### Setup

In [ ]:
# !pip install azure-mgmt-cognitiveservices==13.6.0

In [16]:
from dotenv import load_dotenv
import os

In [17]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient
from azure.mgmt.cognitiveservices.models import Account, Sku
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence.models import AnalyzeResult


In [18]:
# Load environment variables from .env file
load_dotenv(override=True)

True

### Create resource

In [19]:
# Set your Azure subscription ID
subscription_id = os.environ.get("AZURE_SUBSCRIPTION_ID")

In [20]:
# Initialize the Azure credentials and management client
credential = DefaultAzureCredential()
client = CognitiveServicesManagementClient(credential, subscription_id)

In [25]:
# Resource group and resource details
resource_group_name = "ep-rg-post-build-ai-foundry"    # Replace with your resource group name
account_name  = "ep-doc-intelligence-standalone"       # Replace with your desired resource name
location = "westeurope"                                # Replace with your desired Azure region

In [26]:
account_parameters = Account(
    location=location,
    kind="FormRecognizer", 
    sku=Sku(name="S0"),
    properties={}
)

In [27]:
poller = client.accounts.begin_create(
    resource_group_name,
    account_name ,
    account_parameters
)
account = poller.result()
print(f"✅ Resource '{account.name}' created in {account.location}")

✅ Resource 'ep-doc-intelligence-standalone' created in westeurope


### Retrieve Keys and Endpoint

In [28]:
# Retrieve keys
keys = client.accounts.list_keys(resource_group_name, account_name)
key1 = keys.key1

# Retrieve endpoint
account = client.accounts.get(resource_group_name, account_name)
endpoint = account.properties.endpoint

print(f"Endpoint: {endpoint}")

Endpoint: https://westeurope.api.cognitive.microsoft.com/


### Create a Doc Intelligence Client

In [29]:
document_intelligence_client = DocumentIntelligenceClient(
    endpoint=endpoint, 
    credential=AzureKeyCredential(key1),
    # api_version="2024-11-30" # v4.0 (default)
)

#### Test the Doc Intelligence client

In [30]:
# Read the local file in binary mode
with open("assets/invoice-logic-apps-tutorial.pdf", "rb") as file:

    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-layout",
        body=file,
    )

    result: AnalyzeResult = poller.result()

In [31]:
if result.tables:
    print(f"I've found {len(result.tables)} tables.")
else:
    print("I haven't found any tables.")

I've found 3 tables.


### Delete the resource

In [32]:
# Delete the resource
delete_poller = client.accounts.begin_delete(resource_group_name, account_name)
delete_poller.result()  # Wait for completion

print(f"🗑️ Resource '{account_name}' deleted successfully.")

🗑️ Resource 'ep-doc-intelligence-standalone' deleted successfully.
